Start by importing all libraries needed

In [10]:
# Holly Chandler
# Masterclass Research Project
# Code to support maths required in the X-Z laplacian for various functions

#import all neccesary libraries and allow use of unicode symbols
import sympy
from sympy import symbols, expand, factor, latex
from sympy import *
init_printing(use_unicode=True)
#import library allowing ext and latex to be on the same line:
from IPython.display import display, Math

In [11]:
#set it so you can have pretty outputs for multiple different functions in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Inistialise symbols and create lists containing all required values

In [29]:
#initialise all the symbols used for each coordinate system
alpha, gamma, beta, x, y, z, r, theta, phi, R, A, B, D, h, I, Bpol  = symbols('alpha gamma beta x y z r theta phi R A B D h_theta I B_pol')

In [13]:
#order for all lists of components follows:

#0 cartesian (xyz), 
#1 cyllindrical (r,theta,z), 
#2 spherical (r,theta,phi), 
#3 toroidal (r,theta,phi), 
#4 Clebsch Field Alligned Coordinates (x,y,z)

#initialsise coordinate system order:
C = ["cartesian (xyz)","cyllindrical (r,theta,z)","spherical (r,theta,phi)","toroidal (r,theta,phi)","Clebsch Field Alligned Coordinates (x,y,z)" ]

#Initialise all the known Jacobians
J = [1, r, r**(2)*sin(theta),-r*(R+r*cos(theta)),h/Bpol]

#initialise the main diagonal metric tensors in order g11,g22,g33 where 1,2,3 refer to the coordinate system variables in order as below
g = [[1,1,1],[1,1/(r**2),1],[1,1/(r**2),1],[1,1/(r**2),1/(R+r*cos(theta))**2],[(R*Bpol)**2, 1/(h)**2,I**2*(R*Bpol)**2+B**2/(R*Bpol)**2]]

#all the coordinate system variables.
coords = [[x,y,z],[r,theta,z],[r,theta,phi],[r,theta,phi],[x,y,z]]


Function that calculates b depending on constants, function, metric tensors and jacobian

In [24]:
#create a function to independently calculate b
def calculate_b(A,D,f,J,gxx,gzz,z_coord,x_coord):
    b = A/J*(diff(J*gxx*diff(f, x_coord),x_coord)+(diff(J*gzz*diff(f, z_coord),z_coord)))+D*f
    return b

Function that calculates the forward laplacian and outputs the results based on coordinate system and variables chosen

In [27]:
#function finds the symbols of the coordinate system and outputs the result of calculating the forward laplacian

def calculateForwardLaplacian(f,coordSystemChoice,xCoordIndex,zCoordIndex):
    #choose x coordinate unit based on coordinate system and which combination
    x_coord = coords[coordSystemChoice][xCoordIndex]
    z_coord = coords[coordSystemChoice][zCoordIndex]
            
    #assign x and z the units found above to ensure that the function inputted is changed to have those units
    x= x_coord
    z= z_coord         
            
    #calculate result of laplacian by calling calculae_b function
    b = calculate_b(A,B,f,J[coordSystemChoice],g[coordSystemChoice][coordChoiceX],g[coordSystemChoice][coordChoiceZ],z_coord,x_coord)
            
    #print out all the values used
    print("Coordinate system choice is", C[coordSystemChoice])
    display(Math(f'Jacobian = {latex(J[coordSystemChoice])}'))
    display(Math(f'X-Coordinate = {latex(x)}'))
    display(Math(f'Z-Coordinate = {latex(z)}'))
    display(Math(f'f = {latex(f)}'))
    display(Math(f'b = {latex(b)}'))
    print ()

Loop that calculates the output of the forward laplacian in all coordinate systems except for Clebsch with all options of general xz for a given function f.

In [30]:
def loopAllCoordinatesAndSystems(f):
    #for each coordinate system all combinations of units are used i.e 01, 12, 20.
    #coordinate systems are shown as above
    #to change the function that is being inputted into the laplacian, you change f which is seen within the for loops

    #for loop to go through each coordinate system
    for i in range(0,4):
        #for loop for x to go through each basis vector
        for j in range (0,3):
            #for loop to ensure that each combination of basis vectors are seen i.e 01,12,21,02,10,20
            for k in range(0,2):
            
                #set the coordinate system and coord choice through for loop
                coordSystemChoice = i
                coordChoiceX = j
            
                #% is modulus which is the remainder when (j+1) is divided by 3. Y goes 1,2,0.
                coordChoiceZ = (j+k+1)%3
            
                #calculate result:
                calculateForwardLaplacian(f,coordSystemChoice,coordChoiceX,coordChoiceZ)
            

Function that calculates the Forward Laplacian for a Clebsch coordinate system where the coordinate dependent variables are considered

In [31]:
def clebschCoordinatesForwardLaplacian(f):
    #set the coordinate system and coord choice through for loop
    coordSystemChoice = 4

    #choose x coordinate unit based on coordinate system and which combination
    x_coord = coords[coordSystemChoice][0]
    z_coord = coords[coordSystemChoice][2]
            
    #assign x and z the units found above to ensure that the function inputted is changed to have those units
    x= x_coord
    z= z_coord        
        
    #NEW STUFF
    #write Jacobian, metric tensors and functions for B and R
    #start with B functions as then, when the Jacobian and metric tensors are applied, the functions are subbed in.

    eps =.1
    r = eps*(0.9 +0.1*x)
    theta = y-pi
    Rxy = 1 + r*cos(theta)
    h=r
    Bpol = 1/r
    Btor = 1/Rxy
    Bxy = sqrt(Bpol**2 + Btor**2)
    JClebsch = h/Bpol
    clebschMetricTensor = [(Rxy*Bpol)**2, 1/(h)**2,I**2*(Rxy*Bpol)**2+Bxy**2/(Rxy*Bpol)**2]

    A = x+y+sin(2*pi*z)
    D = 1
        
    #calculate result of laplacian by calling calculae_b function
    b = calculate_b(A,D,f,JClebsch,clebschMetricTensor[0],clebschMetricTensor[2],z_coord,x_coord)
            
    #print out all the values used
    print("Coordinate system choice is Clebsch")
    print("Facts about the system")
    display(Math(f'X-Coordinate = {latex(x)}'))
    display(Math(f'Z-Coordinate = {latex(z)}'))
    display(Math(f'Jacobian = {latex(J[coordSystemChoice])} = {latex(JClebsch)}'))
    display(Math(f'Bpol = {latex(Bpol)}'))
    display(Math(f'Btor = {latex(Btor)}'))
    display(Math(f'Bxy = {latex(Bxy)}'))
    display(Math(f'gxx = {latex(clebschMetricTensor[0])}'))
    display(Math(f'gzz = {latex(clebschMetricTensor[2])}'))
    print("Inputted function")
    display(Math(f'f = {latex(f)}'))
    print("Result")
    display(Math(f'b = {latex(b)}'))
    print("result that is copiable to c++ NOTE: change  ** to pow(a,b) to avoid errors in c++")
    print(b)
    print ()